In [3]:
%load_ext autoreload
%autoreload 2

import os
# os.environ['HF_HOME'] = os.environ.get('SCRATCH') + '/iaifi_lab/Lab/ericjm/.cache/huggingface'
os.environ['HF_HOME'] = '/om/user/ericjm/.cache/huggingface'
import json
import argparse
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForCausalLM,
)
from pathlib import Path

from narrow.modeling import convert_pruned_to_variable_size

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'narrow'

In [7]:
SCRATCH = os.environ.get('SCRATCH')
model_dir = os.path.join(SCRATCH, 'iaifi_lab/Lab/ericjm/narrow/pruneandtrain01/n0.80_r0.80/checkpoint-20000')
model = AutoModelForCausalLM.from_pretrained(model_dir)

In [8]:
pruned_model = convert_pruned_to_variable_size(model)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [12]:
# create an input object
input_obj = tokenizer("Hello, how are you?", return_tensors="pt")

# run each model on the input object
baseline_output = model(input_obj["input_ids"])
pruned_output   = pruned_model(input_obj["input_ids"])

In [15]:
tokenizer.decode(input_obj.input_ids[0, 0])

'<|begin_of_text|>'

In [18]:
baseline_output.logits

tensor([[[ 9.3774,  8.1457, 14.4361,  ..., -4.6480, -4.6479, -4.6481],
         [ 9.4960,  3.8903,  4.7837,  ..., -7.2911, -7.2912, -7.2913],
         [ 5.6897,  1.6978,  2.1098,  ..., -6.6646, -6.6646, -6.6647],
         ...,
         [ 5.7636,  2.8113,  1.0498,  ..., -6.6710, -6.6713, -6.6708],
         [ 8.9889,  3.9790,  1.7644,  ..., -7.4253, -7.4253, -7.4250],
         [ 8.3706,  3.4622,  5.3251,  ..., -7.1916, -7.1918, -7.1914]]],
       grad_fn=<UnsafeViewBackward0>)

In [19]:
pruned_output.logits

tensor([[[ 3.6760,  3.3142,  6.1423,  ..., -1.9703, -1.9702, -1.9703],
         [ 5.4280,  3.8526,  3.9846,  ..., -4.8080, -4.8081, -4.8082],
         [ 5.1023,  3.6254,  3.5077,  ..., -4.6185, -4.6185, -4.6186],
         ...,
         [ 5.2408,  2.5246,  2.5794,  ..., -5.2450, -5.2451, -5.2452],
         [ 5.2377,  2.4547,  2.4712,  ..., -5.3923, -5.3925, -5.3925],
         [ 5.2791,  1.9092,  2.3428,  ..., -4.9808, -4.9809, -4.9809]]],
       grad_fn=<UnsafeViewBackward0>)

In [25]:
model.lm_head.weight.data

tensor([[ 0.0000,  0.0173,  0.0277,  ..., -0.0000, -0.0429, -0.0000],
        [ 0.0000, -0.0265,  0.0265,  ..., -0.0000,  0.0026, -0.0000],
        [ 0.0000,  0.0209,  0.0192,  ...,  0.0000, -0.0059,  0.0000],
        ...,
        [ 0.0000,  0.0220, -0.0299,  ..., -0.0000, -0.0031,  0.0000],
        [ 0.0000,  0.0220, -0.0299,  ..., -0.0000, -0.0031,  0.0000],
        [ 0.0000,  0.0221, -0.0299,  ..., -0.0000, -0.0031,  0.0000]])

In [27]:
from nnsight import NNsight

In [28]:
nnmodel = NNsight(model)
nnpruned_model = NNsight(pruned_model)

In [31]:
with nnmodel.trace(input_obj["input_ids"]):
    emb_out = nnmodel.model.embed_tokens.output.save()
with nnpruned_model.trace(input_obj["input_ids"]):
    emb_out_pruned = nnpruned_model.model.embed_tokens.output.save()

In [32]:
emb_out

tensor([[[ 0.0000,  0.0030, -0.0042,  ..., -0.0000,  0.0012,  0.0000],
         [ 0.0000,  0.0043,  0.0247,  ...,  0.0000,  0.0334, -0.0000],
         [ 0.0000, -0.0326,  0.0655,  ..., -0.0000, -0.0248, -0.0000],
         ...,
         [-0.0000,  0.0339,  0.0362,  ..., -0.0000, -0.0167,  0.0000],
         [-0.0000,  0.0451,  0.0511,  ...,  0.0000, -0.0358, -0.0000],
         [-0.0000, -0.0218,  0.0227,  ..., -0.0000, -0.0221,  0.0000]]],
       grad_fn=<EmbeddingBackward0>)

In [33]:
emb_out_pruned

tensor([[[ 0.0030, -0.0042,  0.0063,  ...,  0.0006,  0.0019,  0.0012],
         [ 0.0043,  0.0247,  0.0219,  ...,  0.0063,  0.0057,  0.0334],
         [-0.0326,  0.0655, -0.0131,  ...,  0.0363,  0.0078, -0.0248],
         ...,
         [ 0.0339,  0.0362,  0.0060,  ..., -0.0021, -0.0072, -0.0167],
         [ 0.0451,  0.0511, -0.0138,  ..., -0.0082,  0.0031, -0.0358],
         [-0.0218,  0.0227,  0.0207,  ...,  0.0183,  0.0318, -0.0221]]],
       grad_fn=<EmbeddingBackward0>)

In [7]:
model.model.layers[0].mlp.gate_proj.weight.data.shape

torch.Size([8192, 2048])

In [6]:
from narrow.modeling import convert_pruned_to_variable_size

In [7]:
new_model = convert_pruned_to_variable_size(model)

AssertionError: Layer 1 has different pruned residual stream dimensions than layer 0

In [22]:
(model.model.layers[0].mlp.gate_proj.weight.sum(dim=1).abs() == 0).nonzero()[:, 0]

tensor([   1,    2,    3,  ..., 8177, 8179, 8186])

In [23]:
(model.model.layers[0].mlp.up_proj.weight.sum(dim=1).abs() == 0).nonzero()[:, 0]

tensor([   1,    2,    3,  ..., 8177, 8179, 8186])

In [28]:
(model.model.layers[0].mlp.down_proj.weight.sum(dim=0).abs() == 0).nonzero()[:, 0]

tensor([   1,    2,    3,  ..., 8177, 8179, 8186])

In [82]:
(model.model.layers[0].mlp.gate_proj.weight.data == 0).all(dim=1).nonzero()[:, 0].tolist()

[1,
 2,
 3,
 8,
 13,
 15,
 17,
 18,
 24,
 27,
 29,
 36,
 37,
 38,
 40,
 44,
 49,
 50,
 52,
 54,
 55,
 58,
 59,
 60,
 63,
 72,
 80,
 81,
 83,
 84,
 85,
 88,
 91,
 94,
 96,
 100,
 102,
 103,
 104,
 109,
 111,
 112,
 115,
 117,
 122,
 123,
 124,
 126,
 129,
 130,
 134,
 136,
 140,
 142,
 144,
 145,
 146,
 147,
 150,
 153,
 156,
 158,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 171,
 180,
 183,
 189,
 192,
 194,
 195,
 198,
 202,
 207,
 209,
 210,
 211,
 213,
 216,
 218,
 219,
 224,
 235,
 239,
 240,
 251,
 256,
 264,
 265,
 267,
 269,
 275,
 280,
 281,
 288,
 294,
 298,
 299,
 308,
 309,
 313,
 315,
 317,
 318,
 320,
 321,
 324,
 326,
 328,
 329,
 330,
 334,
 336,
 341,
 350,
 351,
 352,
 354,
 356,
 357,
 362,
 363,
 364,
 368,
 371,
 373,
 382,
 384,
 390,
 394,
 400,
 405,
 407,
 408,
 409,
 411,
 415,
 417,
 419,
 422,
 423,
 427,
 429,
 430,
 431,
 440,
 444,
 445,
 450,
 455,
 456,
 458,
 460,
 463,
 465,
 467,
 472,
 473,
 475,
 480,
 485,
 487,
 491,
 492,
 493,
 494,
 497,
 498,
 

In [43]:
model.model.embed_tokens.weight.shape

torch.Size([128256, 2048])

In [46]:
type(model)

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [47]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "/n/netscratch/iaifi_lab/Lab/ericjm/narrow/trainwhilepruning01/both/checkpoint-15000",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "float32",
  "transformers_version": "4.49.0",
  "use_cache": true,
  "vocab_size": 128256
}

In [70]:
import torch.nn as nn
from transformers.models.llama.modeling_llama import (
    LlamaMLP, 
    LlamaAttention, 
    LlamaRMSNorm, 
    LlamaRotaryEmbedding, 
    LlamaDecoderLayer, 
    LlamaModel, 
    LlamaForCausalLM,
    LlamaPreTrainedModel,
    ACT2FN
)
from transformers.models.llama.configuration_llama import LlamaConfig

In [71]:
class VariableSizeLlamaConfig(LlamaConfig):
    r"""
    This is a subclass of LlamaConfig that allows for variable intermediate sizes across layers.

    Args:
        intermediate_size (`int` or `List[int]`, *optional*, defaults to 11008):
            Dimension of the MLP representations. Can be a single integer (all layers have the same size) or a list of 
            integers (one per layer). If a list is provided, its length must match `num_hidden_layers`.
        
        # ... all other args from LlamaConfig ...
    """

    model_type = "variable_size_llama"

    def __init__(
        self,
        vocab_size=32000,
        hidden_size=4096,
        intermediate_size=11008,
        num_hidden_layers=32,
        num_attention_heads=32,
        num_key_value_heads=None,
        hidden_act="silu",
        max_position_embeddings=2048,
        initializer_range=0.02,
        rms_norm_eps=1e-6,
        use_cache=True,
        pad_token_id=None,
        bos_token_id=1,
        eos_token_id=2,
        pretraining_tp=1,
        tie_word_embeddings=False,
        rope_theta=10000.0,
        rope_scaling=None,
        attention_bias=False,
        attention_dropout=0.0,
        mlp_bias=False,
        head_dim=None,
        **kwargs,
    ):
        # Process the intermediate_size parameter
        if isinstance(intermediate_size, list):
            if len(intermediate_size) != num_hidden_layers:
                raise ValueError(
                    f"If intermediate_size is a list, it must have {num_hidden_layers} elements "
                    f"(one per layer), but got {len(intermediate_size)} elements."
                )
            self.intermediate_sizes = intermediate_size
            # Use the first value for compatibility with parent class
            intermediate_size_for_parent = intermediate_size[0]
        else:
            # If it's a single integer, create a list with the same value for all layers
            self.intermediate_sizes = [intermediate_size] * num_hidden_layers
            intermediate_size_for_parent = intermediate_size

        # Call the parent class constructor with the processed intermediate_size
        super().__init__(
            vocab_size=vocab_size,
            hidden_size=hidden_size,
            intermediate_size=intermediate_size_for_parent,  # Use the first value or original value
            num_hidden_layers=num_hidden_layers,
            num_attention_heads=num_attention_heads,
            num_key_value_heads=num_key_value_heads,
            hidden_act=hidden_act,
            max_position_embeddings=max_position_embeddings,
            initializer_range=initializer_range,
            rms_norm_eps=rms_norm_eps,
            use_cache=use_cache,
            pad_token_id=pad_token_id,
            bos_token_id=bos_token_id,
            eos_token_id=eos_token_id,
            pretraining_tp=pretraining_tp,
            tie_word_embeddings=tie_word_embeddings,
            rope_theta=rope_theta,
            rope_scaling=rope_scaling,
            attention_bias=attention_bias,
            attention_dropout=attention_dropout,
            mlp_bias=mlp_bias,
            head_dim=head_dim,
            **kwargs,
        )

class VariableSizeLlamaMLP(LlamaMLP):
    def __init__(self, config, layer_idx):
        # Don't call super().__init__(config) since we want to override the intermediate_size
        nn.Module.__init__(self)
        self.config = config
        self.hidden_size = config.hidden_size
        
        # Use the layer-specific intermediate size
        self.intermediate_size = config.intermediate_sizes[layer_idx]
        
        self.gate_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias=config.mlp_bias)
        self.up_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias=config.mlp_bias)
        self.down_proj = nn.Linear(self.intermediate_size, self.hidden_size, bias=config.mlp_bias)
        self.act_fn = ACT2FN[config.hidden_act]


class VariableSizeLlamaDecoderLayer(LlamaDecoderLayer):
    def __init__(self, config: LlamaConfig, layer_idx: int):
        super(LlamaDecoderLayer, self).__init__()  # Call nn.Module.__init__()
        self.hidden_size = config.hidden_size
        
        self.self_attn = LlamaAttention(config=config, layer_idx=layer_idx)
        
        # Use the variable size MLP
        self.mlp = VariableSizeLlamaMLP(config, layer_idx)
        
        self.input_layernorm = LlamaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.post_attention_layernorm = LlamaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)


class VariableSizeLlamaModel(LlamaModel):
    def __init__(self, config: LlamaConfig):
        super(LlamaModel, self).__init__(config)  # Call PreTrainedModel.__init__
        self.padding_idx = config.pad_token_id
        self.vocab_size = config.vocab_size

        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size, self.padding_idx)
        
        # Use variable-size decoder layers
        self.layers = nn.ModuleList(
            [VariableSizeLlamaDecoderLayer(config, layer_idx) for layer_idx in range(config.num_hidden_layers)]
        )
        
        self.norm = LlamaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.rotary_emb = LlamaRotaryEmbedding(config=config)
        self.gradient_checkpointing = False

        # Initialize weights and apply final processing
        self.post_init()


class VariableSizeLlamaForCausalLM(LlamaForCausalLM):
    config_class = VariableSizeLlamaConfig
    
    def __init__(self, config):
        # Don't call super().__init__(config) to avoid initializing the standard LlamaModel
        LlamaPreTrainedModel.__init__(self, config)
        
        self.model = VariableSizeLlamaModel(config)
        self.vocab_size = config.vocab_size
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        # Initialize weights and apply final processing
        self.post_init()


In [72]:
new_config = model.config
new_config.intermediate_sizes = [model.config.intermediate_size] * model.config.num_hidden_layers

In [73]:
new_config = VariableSizeLlamaConfig(**new_config.to_dict())

In [74]:
new_model = VariableSizeLlamaForCausalLM(new_config)

In [75]:
new_model

VariableSizeLlamaForCausalLM(
  (model): VariableSizeLlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x VariableSizeLlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): VariableSizeLlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): Ll